## 4. Cascadia: Positional Embedding of Augmented Spectra

### 4.1 Motivation

* Required Reading: https://www.nature.com/articles/s41592-025-02718-y
* Relevant signal spans multiple spectra: across retention time (RT) and between MS1 and MS2 levels.
* A richer representation is needed to capture this context.

### 4.2 2D Positional Encoding

* Represent each peak as a tuple: (*m/z*, intensity, RT offset, MS level).
* Apply **2D sinusoidal embeddings** to jointly encode *m/z* and RT.
* Concatenate with learned embeddings for intensity and spectrum level.
* Output: a **512-dimensional embedding** for each peak in the augmented spectrum.


### Data-Dependent Acquisition (DDA) vs. Data-Independent Acquisition (DIA)
<img src='DDA vs. DIA.png' width=500>

**DDA:** Each fragmentation spectrum corresponds to a single peptide.

At every MS1 scan, the instrument selects the top k most intense precursor ions. Only these precursors are isolated, fragmented, and recorded as MS2 spectra This produces clean MS2 spectra, each representing one peptide. But the data is incomplete becuase of the precursor selection. 

**DIA:** A single peptide is reconstructed from multiple time-adjacent spectra collected across wide, overlapping m/z windows.

Instead of selecting specific precursors, DIA fragments every ion within a set of predefined m/z windows. These windows often overlap to avoid losing peptides at boundaries. A single peptide typically contributes fragments to many consecutive MS2 spectra.

### Cascadia's Architecture
<img src='Cascadia Schematic.webp' width=900>

Cascadia builds off of Casanovo’s core architecture, predicting one amino acid at a time using a Transformer decoder. But, instead of relying on a single encoded MS2 spectrum, Cascadia incorporates multiple sources of spectral evidence (the augmented spectra), allowing it to decode a richer representation. The encoded peak representations, embedded precursor information, and the previouly predicted amino acids in the peptide (P, E, P), are fed into the transformer decoder to predict the next amino acid. Linear and Softmax layers are used once again for each peak, this time to determine if the peaks match a b or y ion for the current peptide.

### Augmented Spectra Introduction
- If we perform a DIA run then, all traces of a peptide is spread across multiple MS2 cycles, and mirrored in MS1 isotopes.
    - Cascadia's solution to this is essentially to look at a given MS2 spectra's _neighboring_ MS2 spectra and all their respective MS1 spectra. Check out the gif below. 

<video controls>
  <source src="AugmentedManim.mp4" type="video/mp4">
</video>

A peptide elutes over about 30 seconds, 